# 🔌 Model Context Protocol (MCP): Connecting AI to the World

**Scenario:** Imagine you're building an AI assistant for a travel agency. Your assistant needs to check real-time weather, search for flights, look up hotel availability, and calculate currency conversions. Each of these requires a different API or service. Managing all these integrations manually would be a nightmare!

This is where **Model Context Protocol (MCP)** comes in. Think of MCP as a universal adapter that lets your AI connect to any tool or service through a standardized interface. Instead of writing custom code for each API, you connect to MCP servers that provide these capabilities.

In this notebook, you'll learn how to:
- Understand what MCP is and why it matters
- Connect to MCP servers
- Use MCP tools with any LLM (not just Gemini!)
- Build practical MCP-powered applications

---

## 🎯 What You'll Build

In this notebook, you will execute external MCP scripts to:
1. **Connect to Servers** - Access MCP servers and their tools
2. **Manual Tool Calling** - Use MCP with ANY LLM provider
3. **Real-world Applications** - Weather bot, data fetcher, multi-tool agent

---

## 📦 Setup: Install Required Packages

We'll use:
- **`litellm`** - AI model interface (works with any provider)
- **`python-dotenv`** - Load API keys
- **`mcp`** - Model Context Protocol library
- **`langchain-google-genai`** - For native Gemini MCP support (optional)

In [1]:
# Install required packages
!pip install -q litellm python-dotenv mcp langchain-google-genai google-generativeai

In [2]:
import os
import sys
from dotenv import load_dotenv

# Load API key from .env file (if it exists)
load_dotenv()

# Configuration
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "gemini/gemini-2.0-flash-exp")

print(f"✅ Using model: {DEFAULT_MODEL}")
print(f"✅ Python Executable: {sys.executable}")

✅ Using model: openrouter/google/gemini-2.0-flash-001
✅ Python Executable: /Users/param/learn/learnwithparam/ai-bootcamp-notebooks/notebooks/ai-foundation-workshop/.venv/bin/python3.14


## 🤔 Part 1: Understanding MCP

**What is MCP?** Model Context Protocol is like a universal remote control for AI. Instead of your AI needing to know how to talk to each specific service (weather API, database, file system), it talks to MCP servers that provide standardized access to these services.

**Why does this matter?**
- **Standardization**: One way to connect to many services
- **Flexibility**: Works with ANY LLM, not just one provider
- **Simplicity**: MCP servers handle the complex API details
- **Reusability**: Same MCP server works with different AI applications

**Real-World Analogy:** Think of MCP like USB ports. Before USB, every device had its own connector. Now, everything uses USB. MCP does the same for AI tools!

---

## 🌤️ Part 2: Connecting to an MCP Server (Weather Example)

We will run the `mcp_weather.py` script. This script connects to the `@philschmid/weather-mcp` server via `npx` and lists the available tools.

**Note:** We run this as a subprocess to ensure a stable `asyncio` event loop.

In [3]:
%%writefile mcp_weather.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import json

# Define the MCP server configuration
# We use 'npx' to run the weather server directly from npm
server_params = StdioServerParameters(
    command="npx",
    args=["-y", "@philschmid/weather-mcp"],
    env=None
)

async def explore_mcp_server():
    print("Connecting to weather server... (this may take a moment to download the npx package)")
    
    # Establish the connection via Stdio (Standard Input/Output)
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the MCP session
            await session.initialize()
            print(f"✓ Connected to MCP weather server!\n")
            
            # List available tools provided by the server
            tools_result = await session.list_tools()
            tools = tools_result.tools
            
            print(f"→ Available Tools:")
            for tool in tools:
                print(f"  • {tool.name}: {tool.description}")
                print(f"    Parameters: {json.dumps(tool.inputSchema, indent=6)}\n")

if __name__ == "__main__":
    asyncio.run(explore_mcp_server())

Overwriting mcp_weather.py


In [4]:
import sys
print(f"Running mcp_weather.py using {sys.executable}...")
!{sys.executable} mcp_weather.py

Running mcp_weather.py using /Users/param/learn/learnwithparam/ai-bootcamp-notebooks/notebooks/ai-foundation-workshop/.venv/bin/python3.14...
Connecting to weather server... (this may take a moment to download the npx package)
✓ Connected to MCP weather server!

→ Available Tools:
  • get_weather_forecast: Retrieves the weather using Open-Meteo API for a given location (city) and a date (yyyy-mm-dd). Returns a list dictionary with the time and temperature for each hour.
    Parameters: {
      "type": "object",
      "properties": {
            "location": {
                  "type": "string",
                  "description": "The city and state, e.g., San Francisco, CA"
            },
            "date": {
                  "type": "string",
                  "description": "the forecasting date for when to get the weather format (yyyy-mm-dd)"
            }
      },
      "required": [
            "location",
            "date"
      ],
      "additionalProperties": false,
      "$sch

### ❓ Discussion

The output above shows the tools available on the server (like `get_weather_forecast`).

---

## 🛠️ Part 3: Manual Tool Calling (Works with ANY LLM)

We will now create an agent script that:
1. Connects to the MCP server.
2. **Injects the current date** into the prompt (Context Grounding).
3. Asks the LLM to pick a tool.
4. Executes the tool and returns the result.

In [5]:
%%writefile mcp_agent.py
import asyncio
import os
import json
from datetime import datetime
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from litellm import completion
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "gemini/gemini-2.0-flash-exp")

server_params = StdioServerParameters(
    command="npx",
    args=["-y", "@philschmid/weather-mcp"],
    env=None
)

async def run_agent():
    print("Starting MCP Agent Workflow...")
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 1. Get Tools
            tools_result = await session.list_tools()
            tools = tools_result.tools
            tools_desc = "\n".join([f"- {t.name}: {t.description} (Params: {json.dumps(t.inputSchema)})" for t in tools])
            
            # 2. Context Grounding: Get Current Date
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M")
            print(f"[System] Current Time: {current_time}")
            
            # 3. Ask AI to pick a tool
            question = "What is the weather in Berlin today?"
            print(f"\nUser Question: {question}")
            
            prompt = f"""System: You are a helpful assistant. The current date and time is {current_time}.
            
Available tools:
{tools_desc}

User question: {question}

Respond ONLY with JSON: {{"tool": "tool_name", "parameters": {{...}}}}"""
            
            response = completion(model=DEFAULT_MODEL, messages=[{"role": "user", "content": prompt}])
            content = response.choices[0].message.content
            
            try:
                # Clean up potential markdown code blocks
                clean_content = content.replace("```json", "").replace("```", "").strip()
                action = json.loads(clean_content)
                
                tool_name = action["tool"]
                tool_args = action["parameters"]
                print(f"\nAI Decided to call: {tool_name} with {tool_args}")
                
                # 4. Call the tool via MCP
                result = await session.call_tool(tool_name, tool_args)
                tool_output = result.content[0].text
                print(f"Tool Output: {tool_output}")
                
                # 5. Final Answer
                final_prompt = f"Question: {question}\nData: {tool_output}\nProvide a natural answer."
                final_res = completion(model=DEFAULT_MODEL, messages=[{"role": "user", "content": final_prompt}])
                print(f"\nFINAL ANSWER: {final_res.choices[0].message.content}")
                
            except Exception as e:
                print(f"Error parsing or executing: {e}")

if __name__ == "__main__":
    asyncio.run(run_agent())

Overwriting mcp_agent.py


In [6]:
# Run the agent script
print(f"Running mcp_agent.py using {sys.executable}...")
!{sys.executable} mcp_agent.py

Running mcp_agent.py using /Users/param/learn/learnwithparam/ai-bootcamp-notebooks/notebooks/ai-foundation-workshop/.venv/bin/python3.14...
Starting MCP Agent Workflow...
[System] Current Time: 2025-11-25 23:06

User Question: What is the weather in Berlin today?

AI Decided to call: get_weather_forecast with {'location': 'Berlin, Germany', 'date': '2025-11-25'}
Tool Output: {"2025-11-25T00:00":1.3,"2025-11-25T01:00":1.3,"2025-11-25T02:00":1.6,"2025-11-25T03:00":1.6,"2025-11-25T04:00":1.7,"2025-11-25T05:00":1.5,"2025-11-25T06:00":1.6,"2025-11-25T07:00":1.5,"2025-11-25T08:00":1.6,"2025-11-25T09:00":1.9,"2025-11-25T10:00":2.4,"2025-11-25T11:00":3.4,"2025-11-25T12:00":4.2,"2025-11-25T13:00":4.7,"2025-11-25T14:00":4.9,"2025-11-25T15:00":4.5,"2025-11-25T16:00":3.9,"2025-11-25T17:00":2.9,"2025-11-25T18:00":2.9,"2025-11-25T19:00":2.6,"2025-11-25T20:00":1.9,"2025-11-25T21:00":1.3,"2025-11-25T22:00":0.7,"2025-11-25T23:00":0.1}

FINAL ANSWER: The temperature in Berlin today, November 25, 2025, r

## 🚀 Next Steps:

- **Build** real API integrations (weather, search, databases)
- **Add** authentication and rate limiting
- **Combine** with structured outputs for better reliability
- **Create** multi-agent systems with specialized tools

---

## 📚 Additional Resources

- [Model Context Protocol (MCP) Documentation](https://modelcontextprotocol.io/)
- [MCP Servers on npm](https://www.npmjs.com/search?q=mcp)
- [Building MCP Servers](https://modelcontextprotocol.io/docs/develop/build-server)

Happy building! 🎉
